In [1]:
import numpy as np
from collections import defaultdict
import pandas as pd
from functools import reduce
import glob
import torch
# import matplotlib.pyplot as plt


In [2]:
result_collect_mae = defaultdict(dict)
result_collect_mse = defaultdict(dict)

models = ["contrast-dtcn-moco"]


In [3]:
# sorted_model = []
# for model in models:
#     for predleng in [24, 48, 168, 336, 720]:
#         sorted_model.append(model+"_" + str(predleng))
        
sorted_out_task = ["ECL", "ETTh1", "ETTm1"]  #, "ETTh2", "ETTm1", "ECL"

In [4]:
def visual_itera(mse_loss, cl_loss, save_path):
    plt.figure()
    plt.plot(mse_loss, label='MSE loss', linewidth=2)
    if len(cl_loss) > 0:
        plt.plot(cl_loss, label='CL loss', linewidth=2)
    plt.legend()
    plt.savefig(save_path, bbox_inches='tight')
def visual_epoch(train_loss, val_loss, test_loss, save_path):
    plt.figure()
    plt.plot(train_loss, label='Training loss', linewidth=2)
    plt.plot(val_loss, label='Validation loss', linewidth=2)
    plt.plot(test_loss, label='Test loss', linewidth=2)
    plt.legend()
    plt.savefig(save_path, bbox_inches='tight')

In [38]:
paths

['./backbone_10l/contrast-tcn-moco_ETTh1_contrastL0.5_mask0.3_l2normTrue_ftM_timeFTrue_mareTrue_sl336_pl336_dm512_el10_avgFalse_cosTrue_augcost_Exp_2',
 './backbone_10l/contrast-tcn-moco_ETTh1_contrastL0.5_mask0.3_l2normTrue_ftM_timeFTrue_mareTrue_sl720_pl720_dm512_el10_avgFalse_cosTrue_augcost_Exp_2',
 './backbone_10l/contrast-tcn-moco_ECL_contrastL0.5_mask0.3_l2normTrue_ftM_timeFTrue_mareTrue_sl48_pl24_dm512_el10_avgFalse_cosTrue_augcost_Exp_1',
 './backbone_10l/contrast-tcn-moco_ETTh2_contrastL0.5_mask0.3_l2normTrue_ftM_timeFTrue_mareTrue_sl96_pl48_dm512_el10_avgFalse_cosTrue_augcost_Exp_0',
 './backbone_10l/contrast-tcn-moco_ETTm1_contrastL0.5_mask0.3_l2normTrue_ftM_timeFTrue_mareTrue_sl672_pl672_dm512_el10_avgFalse_cosTrue_augcost_Exp_0',
 './backbone_10l/contrast-tcn-moco_ETTh1_contrastL0.5_mask0.3_l2normTrue_ftM_timeFTrue_mareTrue_sl168_pl168_dm512_el10_avgFalse_cosTrue_augcost_Exp_0',
 './backbone_10l/contrast-tcn-moco_ETTm1_contrastL0.5_mask0.3_l2normTrue_ftM_timeFTrue_mareTru

In [18]:
all_df = []
root_path = "./backbone_10l/"
var_res = "multivar"
weight = "0.0"
vis = False
for weight in ["0.5"]:
    for model in ["-tcn-moco-hcl_", "-dtcn-moco-hcl_", "-lstm-moco-hcl_", "informer-moco-hcl_"]: #"lstm", "tcn"
        #"-tcn-moco-hcl_", "-dtcn-moco-hcl_", "-lstm-moco-hcl_", "informer-moco-hcl_"
        #"-tcn-moco_", "-dtcn-moco_", "-lstm-moco_", "informer-moco_"
        print(model)
        mask_rule = model

        paths = glob.glob(root_path + "/*")

        paths = [x for x in paths if "csv" not in x]
        paths = [x for x in paths if model in x]
        paths = [x for x in paths if "contrastL"+weight in x]
#         paths = [x for x in paths if "cosFalse" in x]
#         paths = [x for x in paths if "mareTrue" in x]
#         paths = [x for x in paths if "cldecayFalse" in x]

        result_collect_mae = defaultdict(dict)
        result_collect_mse = defaultdict(dict)

        for p in paths:
            file = p.split("/")[2]

            if "ftS" in file:
                var_type = "univar"
            else:
                var_type = "multivar"

            if var_type == var_res:
                pl = int(file.split("_pl")[1].split("_")[0])
                data = file.split("_")[1]
                try:
                    metric = np.load(p + "/metrics.npy")

                    mae = metric[0]
                    mse = metric[1]

                    if pl in result_collect_mae[data].keys():
                        result_collect_mae[data][pl].append(mae)
                        result_collect_mse[data][pl].append(mse)
                    else:
                        result_collect_mae[data][pl] = [mae]
                        result_collect_mse[data][pl] = [mse]

                    if "Exp_0" in p:
                        loggs = torch.load(p + "/training_log.pt")
                        itera_train_loss_mse = loggs["itera_train_loss_mse"]
                        itera_train_loss_cl = loggs["itera_train_loss_cl"]
                        epoch_train_loss = loggs["epoch_train_loss"]
                        epo_eval_loss = loggs["epo_eval_loss"]
                        epo_test_loss = loggs["epo_test_loss"]
                        num_step = loggs["num_step"]
                        total_step = len(itera_train_loss_mse)

                        if vis:
                            visual_itera(itera_train_loss_mse, itera_train_loss_cl, "./hyper_results/visualization/iteration_"+p.split("/")[-1]+".jpg")
                            visual_itera(itera_train_loss_mse[int(total_step*0.1):], itera_train_loss_cl[int(total_step*0.1):], "./hyper_results/visualization/zoom_iteration_"+p.split("/")[-1]+".jpg")
                            visual_epoch(epoch_train_loss, epo_eval_loss, epo_test_loss, "./hyper_results/visualization/epoch_"+p.split("/")[-1]+".jpg")

                except:
                    print(p)
                    continue

        for data in result_collect_mae.keys():
            for pl in result_collect_mae[data].keys():
                print(data,pl)
                print(len(result_collect_mae[data][pl]))
        #         print(result_collect_mae[data][pl])

                result_collect_mae[data][pl] = sum(result_collect_mae[data][pl]) / len(result_collect_mae[data][pl])
                result_collect_mse[data][pl] = sum(result_collect_mse[data][pl]) / len(result_collect_mse[data][pl])

        result_mae = []
        result_mse = []
        for data in sorted_out_task:
            for pl in result_collect_mae[data].keys():

                result_mae.append([data+"-"+str(pl).zfill(3), result_collect_mae[data][pl]])
                result_mse.append([data+"-"+str(pl).zfill(3), result_collect_mse[data][pl]])


        res_df_mae = pd.DataFrame(result_mae, columns = ["data", "mae_"+model+weight]).sort_values(by="data")
        res_df_mse = pd.DataFrame(result_mse, columns = ["data", "mse_"+model+weight]).sort_values(by="data")

        res_df = res_df_mse.merge(res_df_mae)

        all_df.append(res_df)

-tcn-moco-hcl_
ETTh1 48
3
ETTh1 24
3
ETTh1 168
3
ETTh1 720
3
ETTh1 336
3
ETTm1 96
3
ETTm1 672
3
ETTm1 24
3
ETTm1 288
3
ETTm1 48
3
ECL 336
3
ECL 720
3
ECL 48
3
ECL 24
3
ECL 168
3
-dtcn-moco-hcl_
ETTm1 48
3
ETTm1 96
3
ETTm1 672
3
ETTm1 24
3
ETTm1 288
3
ECL 720
3
ECL 336
3
ECL 24
3
ECL 168
3
ECL 48
3
ETTh1 24
3
ETTh1 336
3
ETTh1 720
3
ETTh1 168
3
ETTh1 48
3
-lstm-moco-hcl_
ETTm1 288
3
ETTm1 48
3
ETTm1 24
3
ETTm1 672
3
ETTm1 96
3
ECL 336
3
ECL 24
3
ECL 168
3
ECL 48
3
ECL 720
3
ETTh1 168
3
ETTh1 24
3
ETTh1 336
3
ETTh1 720
3
ETTh1 48
3
informer-moco-hcl_
ETTm1 96
3
ETTm1 24
3
ETTm1 288
3
ETTm1 48
3
ETTm1 672
3
ETTh1 168
3
ETTh1 48
3
ETTh1 336
3
ETTh1 720
3
ETTh1 24
3
ECL 24
3
ECL 48
3
ECL 720
3
ECL 336
3
ECL 168
3


In [19]:
final_df = reduce(lambda df1,df2: pd.merge(df1,df2,on='data',how="outer"), all_df)
final_df.loc[final_df.shape[0]] = final_df.mean()
final_df["data"][final_df.shape[0]-1] = "mean"
final_df.to_csv( "./" + var_res +"avg.csv")

/tmp/ipykernel_1794980/851898978.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["data"][final_df.shape[0]-1] = "mean"


In [20]:
final_df

,data,mse_-tcn-moco-hcl_0.5,mae_-tcn-moco-hcl_0.5,mse_-dtcn-moco-hcl_0.5,mae_-dtcn-moco-hcl_0.5,mse_-lstm-moco-hcl_0.5,mae_-lstm-moco-hcl_0.5,mse_informer-moco-hcl_0.5,mae_informer-moco-hcl_0.5
0,ECL-024,0.258882,0.349693,0.239820,0.343488,0.301900,0.372060,0.253145,0.352883
1,ECL-048,0.290773,0.371003,0.264222,0.358086,0.383001,0.416092,0.284084,0.370019
2,ECL-168,0.317811,0.396839,0.295855,0.387371,0.331364,0.398207,0.269783,0.361496
3,ECL-336,0.296460,0.376030,0.279746,0.362775,0.341825,0.400413,0.290070,0.373100
4,ECL-720,0.360655,0.420108,0.306225,0.382554,0.396271,0.433341,0.359540,0.413780
5,ETTh1-024,0.506304,0.499370,0.456644,0.467657,0.926634,0.755660,0.568307,0.573989
6,ETTh1-048,0.511022,0.515724,0.571140,0.571893,1.028993,0.804004,0.876335,0.747840
7,ETTh1-168,0.770021,0.679741,0.850964,0.713791,1.022470,0.802974,0.938061,0.731039
8,ETTh1-336,0.961604,0.761295,1.271452,0.887416,1.063993,0.822034,1.128746,0.836260
9,ETTh1-720,0.938291,0.757558,1.414276,0.954462,1.135398,0.843926,1.001755,0.804307


In [32]:
loggs.keys()

dict_keys(['itera_train_loss_mse', 'itera_train_loss_cl', 'epoch_train_loss', 'epo_eval_loss', 'epo_test_loss', 'num_step'])

In [4]:
# dfs_mae = []
# dfs_mse = []

# for run in [0, 1, 2]:
#     for predleng in [24, 48, 168, 336, 720]:
#         for data in ["ETTh1"]:#["ECL", "ETTm2", "Exchange", "traffic", "weather"]:
#             for model in models:
#                 if data == "ECL":
#                     file_name = "custom"
#                 else:
#                     file_name = data
                    
#                 folder_name = "./results/{}_{}_contrastL0.5_ftM_s168_ll68_pl{}_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_Exp_{}".format( model, data, file_name, str(predleng), str(run))
                
# #                 try:
#                 metric = np.load(folder_name + "/metrics.npy")
#                 mae = metric[0]
#                 mse = metric[1]
#                 result_collect_mae[data][model+"_" + str(predleng)] = mae
#                 result_collect_mse[data][model+"_" + str(predleng)] = mse
# #                 except:
# #                 print(folder_name)
# #                 break
    
#     result_mae = np.zeros((len(sorted_out_task), len(sorted_model)))
#     result_mse = np.zeros((len(sorted_out_task), len(sorted_model)))

#     for i, task in enumerate(sorted_out_task):
#         for j, model in enumerate(sorted_model):
#             try:
#                 result_mae[i][j] = result_collect_mae[task][model]
#                 result_mse[i][j] = result_collect_mse[task][model]
#             except:
#                 pass

#     res_df_mae = pd.DataFrame(result_mae, columns=sorted_model)
#     res_df_mse = pd.DataFrame(result_mse, columns=sorted_model)
#     # res_df["best_model"] = res_df.idxmax(axis=1)

# #     res_df.loc['mean'] = res_df.mean()
#     res_df_mae.insert(loc=0, column='task', value=sorted_out_task)
#     res_df_mse.insert(loc=0, column='task', value=sorted_out_task)
#     dfs_mae.append(res_df_mae)
#     dfs_mse.append(res_df_mse)
            

In [5]:
# df_concat_mae = pd.concat(dfs_mae)
# df_avg_mae = df_concat_mae.groupby(level=0).mean()

# df_avg_mae.insert(loc=0, column='task', value=sorted_out_task)
# df_avg_mae.to_csv("./avg_mae.csv", index=False)

In [120]:
# df_concat_mse = pd.concat(dfs_mse)
# df_avg_mse = df_concat_mse.groupby(level=0).mean()

# df_avg_mse.insert(loc=0, column='task', value=sorted_out_task)
# df_avg_mse.to_csv("./avg_mse.csv", index=False)

In [121]:
# df_avg_mae

In [122]:
# df_concat_mae = pd.concat(dfs_mae)
# df_std_mae = df_concat_mae.groupby(level=0).std()

# df_std_mae.insert(loc=0, column='task', value=sorted_out_task)
# df_std_mae.to_csv("./std_mae.csv", index=False)

In [123]:
# df_concat_mse = pd.concat(dfs_mse)
# df_std_mse = df_concat_mse.groupby(level=0).std()

# df_std_mse.insert(loc=0, column='task', value=sorted_out_task)
# df_std_mse.to_csv("./std_mse.csv", index=False)

In [124]:
# df_std_mae

In [125]:
# for run in [0, 1, 2]:
#     for predleng in [24, 48, 168, 336, 720]:
#         for data in ["ETTh1"]:#["ECL", "ETTm2", "Exchange", "traffic", "weather"]:
#             for model in models:
#                 if data == "ECL":
#                     file_name = "custom"
#                 else:
#                     file_name = data
                    
#                 folder_name = "./results/{}_{}_contrastL0.5_ftM_s168_ll68_pl{}_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_Exp_{}".format( model, data, file_name, str(predleng), str(run))
                
# #                 try:
#                 metric = np.load(folder_name + "/metrics.npy")
#                 mae = metric[0]
#                 mse = metric[1]
#                 result_collect_mae[data][model+"_" + str(predleng)] = mae
#                 result_collect_mse[data][model+"_" + str(predleng)] = mse
# #                 except:
# #                 print(folder_name)
# #                 break
    
#     result_mae = np.zeros((len(sorted_out_task), len(sorted_model)))
#     result_mse = np.zeros((len(sorted_out_task), len(sorted_model)))

#     for i, task in enumerate(sorted_out_task):
#         for j, model in enumerate(sorted_model):
#             try:
#                 result_mae[i][j] = result_collect_mae[task][model]
#                 result_mse[i][j] = result_collect_mse[task][model]
#             except:
#                 pass

#     res_df_mae = pd.DataFrame(result_mae, columns=sorted_model)
#     res_df_mse = pd.DataFrame(result_mse, columns=sorted_model)
#     # res_df["best_model"] = res_df.idxmax(axis=1)

# #     res_df.loc['mean'] = res_df.mean()
#     res_df_mae.insert(loc=0, column='task', value=sorted_out_task)
#     res_df_mse.insert(loc=0, column='task', value=sorted_out_task)
#     dfs_mae.append(res_df_mae)
#     dfs_mse.append(res_df_mse)
            

In [13]:
# res_df_mse.to_csv("./hyper_results/"+var_res+mask_rule+"avg_mse.csv")
# res_df_mae.to_csv("./hyper_results/"+var_res+mask_rule+"avg_mae.csv")
# res_df.to_csv("./hyper_results/"+var_res+mask_rule+"avg.csv")